In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_complaint_columns
from lib.textract import parse_textract_datetime
from clean.baton_rouge_pd_cprr import clean_18

In [2]:
df = clean_18()
df

standardize_from_lookup_table: unmatched sequences:
  {'', 'awol ', ', ', ': ', '; ', ' (60 day rule)', 'command of temper (', ' & '}


,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,rule_code,paragraph_code,...,receive_year,receive_month,receive_day,occur_year,occur_month,occur_day,tracking_number,uid,agency,data_production_year
0,administrative review,exonerated,exonerated,P1254,cib street crimes unit,barcelona,jesse,r,3:17,40,...,2018,1,2,2017,12,30,2018-002,4990106e2df85caab5910fe45de338ef,Baton Rouge PD,2017
1,administrative review,not sustained,not sustained,P10280,cib narcotics,acree,jason,k,3:17,40,...,2018,1,9,2018,1,9,2018-004,ddbd2be4c2f4c79dfd5aaa4b4e49212d,Baton Rouge PD,2018
2,office investigation,resigned,office investigation,P10476,op service comm center,rimes,melissa,j,2:4,66,...,2018,1,9,2017,12,3,2018-003,0383a0f9c3222535e9f91e316d9fa9a5,Baton Rouge PD,2017
3,office investigation,office investigation,office investigation,P10476,op service comm center,rimes,melissa,j,2:4,66,...,2018,1,9,2017,12,3,2018-003,0383a0f9c3222535e9f91e316d9fa9a5,Baton Rouge PD,2017
4,administrative review,not sustained,not sustained,P10578,cib cib,hamadeh,yuseff,g,3:20,53,...,2018,1,10,2018,1,10,2018-005,9d1a70ded8a2660295b9a4ad9cb357a5,Baton Rouge PD,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,3:17,40,...,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,Baton Rouge PD,2018
145,administrative review,letter of reprimand,not sustained,P1245,patrol 2nd district,clark,kenneth,m,3:18,43,...,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,Baton Rouge PD,2018
146,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,2:12,21,...,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,Baton Rouge PD,2018
147,administrative review,not sustained,not sustained,P10639,patrol 1st district,robertson,jason,r,3:17,40,...,2018,1,26,2018,1,26,2018-010,adcf8ba6410ae8beb2311493fb0a8f29,Baton Rouge PD,2018


In [3]:
df[["first_name", "middle_initial", "last_name"]]

,first_name,middle_initial,last_name
0,jesse,r,barcelona
1,jason,k,acree
2,melissa,j,rimes
3,melissa,j,rimes
4,yuseff,g,hamadeh
...,...,...,...
144,kenneth,m,clark
145,kenneth,m,clark
146,kenneth,m,clark
147,jason,r,robertson


In [6]:
df.middle_initial.unique()

array(['r', 'k', 'j', 'g', 't', '', 'h', 'd', 'm', 'a', 's', 'n/a', 'e',
       'b', 'cameron', 'wayne', 'w', 'o', 'p', 'n', 'l', 'james',
       'leduff', 'c'], dtype=object)